<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Import packages</a></span></li><li><span><a href="#Custom-functions" data-toc-modified-id="Custom-functions-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>Custom functions</a></span></li><li><span><a href="#Set-request-parameters" data-toc-modified-id="Set-request-parameters-0.0.3"><span class="toc-item-num">0.0.3&nbsp;&nbsp;</span>Set request parameters</a></span></li><li><span><a href="#Crawl-main-page" data-toc-modified-id="Crawl-main-page-0.0.4"><span class="toc-item-num">0.0.4&nbsp;&nbsp;</span>Crawl main page</a></span></li><li><span><a href="#Crawl-each-sub-page" data-toc-modified-id="Crawl-each-sub-page-0.0.5"><span class="toc-item-num">0.0.5&nbsp;&nbsp;</span>Crawl each sub-page</a></span></li><li><span><a href="#Class" data-toc-modified-id="Class-0.0.6"><span class="toc-item-num">0.0.6&nbsp;&nbsp;</span>Class</a></span></li><li><span><a href="#Chotot" data-toc-modified-id="Chotot-0.0.7"><span class="toc-item-num">0.0.7&nbsp;&nbsp;</span>Chotot</a></span></li></ul></li></ul></li><li><span><a href="#Set-main-url-page" data-toc-modified-id="Set-main-url-page-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set main url page</a></span></li><li><span><a href="#Set-driver-for-selenium" data-toc-modified-id="Set-driver-for-selenium-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set driver for selenium</a></span></li><li><span><a href="#Retrieve-webpage" data-toc-modified-id="Retrieve-webpage-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Retrieve webpage</a></span></li><li><span><a href="#Parse-retrieved-page" data-toc-modified-id="Parse-retrieved-page-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Parse retrieved page</a></span></li><li><span><a href="#Retrieve-all-page-links" data-toc-modified-id="Retrieve-all-page-links-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Retrieve all page links</a></span></li><li><span><a href="#Cycle-through-each-page-link" data-toc-modified-id="Cycle-through-each-page-link-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Cycle through each page link</a></span></li><li><span><a href="#posting_info-=-BeautifulSoup(driver.page_source,-'html.parser')" data-toc-modified-id="posting_info-=-BeautifulSoup(driver.page_source,-'html.parser')-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>posting_info = BeautifulSoup(driver.page_source, 'html.parser')</a></span></li></ul></div>

### Import packages

In [1]:
import time, re, json, requests, pandas as pd, numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

### Custom functions

In [2]:
def process_datetime(soup, class_):
    for dt in soup.find('li', class_=class_):
        dt_str = dt.text
        if 'at' in dt_str:
            dt_str = dt_str.replace('at', '')
        dt_str = dt_str.split()
        return datetime.timestamp(datetime.strptime(dt_str[0], '%d/%m/%y'))
    
def get_thread_id(soup):
    for data in soup.find_all('link', rel='canonical'):
        return data.get('href').split('.')[-1].replace('/','')
    
def get_seller_info(soup):
    tmp = []
    for info in soup.find('div', class_='threadview-header--seller'):
        tmp.append(info)

    tmp[1] = tmp[1].span.text
    if tmp[3].span:
        tmp[3] = datetime.timestamp(datetime.strptime(tmp[3].span.text, '%d/%m/%y'))
    else:
        tmp[3] = tmp[3].abbr['data-time']
    tmp[5] = tmp[5].dd.text
    tmp[7] = tmp[7].dd.text
    return tmp

### Set request parameters

In [3]:
# Marketplace page
url = 'https://nhattao.com/f/can-mua-trao-doi.589/'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) \
                AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

### Crawl main page

In [4]:
# Marketplace page
url = 'https://nhattao.com/f/can-mua-trao-doi.589/'

# Create request session
r = requests.Session()

# Retrieve webpage
pages = r.get(url, headers=headers)

# Parse webpage contents
pages = BeautifulSoup(pages.text, 'html.parser')

In [5]:
# Determine maximum number of webpages
no_pages = [int(link.text) for link in pages.find_all('a', \
                attrs={'class': '', 'href': re.compile(r'.*?type=recent&search.*')}) \
                                                      if link.text.isdigit()]
max_pages = max(no_pages)
print('Number of pages on website: {}'.format(max_pages))

Number of pages on website: 34


### Crawl each sub-page

In [6]:
# For each page, retrieve the list of postings
combined = []
for i in range(1, max_pages+1):
    print('Processing page {} of {}'.format(i, max_pages))
    listing_links = []
    url = 'https://nhattao.com/f/can-mua-trao-doi.589/page-{}?type=recent&search_id=122532862&order=up_time&direction=desc'.format(i) 
    page_listings = r.get(url, headers=headers)
    listings = BeautifulSoup(page_listings.text, 'html.parser')
    
    # Grab all listing links from each page
    for link in listings.find_all('a', attrs={'href': re.compile(r'threads/.*'), \
                                              'class': 'Nhattao-CardItem--image'}):
        listing_links.append('https://nhattao.com/'+link.get('href'))
        
    # Retrieve information about each listing
    for j in range(len(listing_links)):
        print('Processing page {} of {}, link {} of {}'.format(max_pages, i, j+1, \
                                                               len(listing_links)))
        print(listing_links[j])
        print('\n')
        
        # Prevent timeout
        time.sleep(5)
        info = r.get(listing_links[j], headers=headers)
        data = BeautifulSoup(info.text, 'html.parser')                
        seller_info = get_seller_info(data)
        if data.find('li', class_='threadview-header--classifiedStatus'):
            condition = data.find('li', class_='threadview-header--classifiedStatus').text
        else:
            condition = 'N/A'
            
        if data.find('li', class_='threadview-header--classifiedLoc'):
            location = data.find('li', class_='threadview-header--classifiedLoc').text.strip()
        else:
            location = 'N/A'
        
        if data.find('span', class_='address'):
            addr = data.find('span', class_='address').text.strip()
        else:
            addr = 'N/A'
            
        if data.find('a', class_='threadview-header--contactPhone'):
            contact = data.find('a', class_='threadview-header--contactPhone').text.strip().replace(' ','')
        else:
            contact = 'N/A'
            
        results = {
            'Thread Link': listing_links[j],
            'Thread ID': get_thread_id(data),
            'Title': data.find('h2').text,
            'Condition': condition,
            'Location': location,
            'Posted Date': process_datetime(data, 'threadview-header--postDate'),
            'Seen': int(data.find('li', class_='threadview-header--viewCount').text.split()[1].replace('.','')),
            'Price': float(data.find('p', class_='threadview-header--classifiedPrice').text.strip().replace(' đ', '').replace('.','')),
            'Contact': contact,
            'Address': addr,
            'Seller': seller_info[1],
            'Date Joined': seller_info[3],
            'No Products': seller_info[5],
            'Likes': seller_info[7]
                 }
        combined.append(results)

print('Writing json file'.format(i))
with open('./nhattao_marketplace.json', 'w') as f:
    json.dump(combined, f)

Processing page 1 of 34
Processing page 1, link 1 of 30
https://nhattao.com/threads/shop-chuyen-thu-mua-may-choi-game-ds-2ds-3ds-psp-vita-ps3-ps4-wiiu-xbox-077-707-1528.7220310/


Processing page 1, link 2 of 30
https://nhattao.com/threads/thu-mua-may-anh-cu-gia-cao-tan-noi-tai-ha-noi.8490455/


Processing page 1, link 3 of 30
https://nhattao.com/threads/can-mua-mot-so-linh-kien-may-tinh-bi-hu-hoac-loi.7296829/


Processing page 1, link 4 of 30
https://nhattao.com/threads/thu-mua-may-game-gia-cao-ps4-ps3-xbox-nintendo-switch-0947700818.6430441/


Processing page 1, link 5 of 30
https://nhattao.com/threads/ban-ten-mien-thegioiruouvang-com.8393484/


Processing page 1, link 6 of 30
https://nhattao.com/threads/ong-kinh-sigma-35mm-f-1-4-dg-hsm-art-for-canon.8524643/


Processing page 1, link 7 of 30
https://nhattao.com/threads/vi-nam.8525203/


Processing page 1, link 8 of 30
https://nhattao.com/threads/giay-nike-womens-air-zoom-vomero-13-running-shoe.8455852/


Processing page 1, link 9 o

Processing page 3, link 9 of 30
https://nhattao.com/threads/in-bao-bi-nilon-dung-gao.8525386/


Processing page 3, link 10 of 30
https://nhattao.com/threads/ban-tai-khoan-google-drive-unlimited-google-drive-khong-gioi-han.8366250/


Processing page 3, link 11 of 30
https://nhattao.com/threads/thanh-long-binh-thuan-gia-si.8495763/


Processing page 3, link 12 of 30
https://nhattao.com/threads/thu-mua-laptop-cu-gia-cao-tphcm-0977419856-nghia.2597794/


Processing page 3, link 13 of 30
https://nhattao.com/threads/dich-vu-thu-mua-may-tinh-laptop-macbook-tai-ha-noi.8490457/


Processing page 3, link 14 of 30
https://nhattao.com/threads/oto-cu-thu-mua-oto-cu-gia-cao-tai-tp-hcm-mua-oto-da-qua-su-dung.3264194/


Processing page 3, link 15 of 30
https://nhattao.com/threads/mua-xe-dap-dien-cu-gia-cao.4403397/


Processing page 3, link 16 of 30
https://nhattao.com/threads/can-mua-con-ps-vita-tv.8423449/


Processing page 3, link 17 of 30
https://nhattao.com/threads/do-choi-xep-hinh-tu-chau-au-an-

Processing page 5, link 17 of 30
https://nhattao.com/threads/do-choi-xep-hinh-tu-chau-au-an-toan-thong-minh-kich-thich-phat-trien-tu-duy-cho-be.8466682/


Processing page 5, link 18 of 30
https://nhattao.com/threads/nhung-mau-thiep-sinh-nhat-vintage-vo-cung-doc-dao-va-la-mat.8524838/


Processing page 5, link 19 of 30
https://nhattao.com/threads/chuyen-thu-mua-laptop-cu-gia-cao-khu-vuc-tphcm.8494482/


Processing page 5, link 20 of 30
https://nhattao.com/threads/the-cao-data-3g-mobifone-tron-goi-1-nam.8517668/


Processing page 5, link 21 of 30
https://nhattao.com/threads/dich-vu-thu-mua-may-tinh-laptop-macbook-tai-ha-noi.8524751/


Processing page 5, link 22 of 30
https://nhattao.com/threads/thu-mua-may-anh-cu-gia-cao-tan-noi-tai-ha-noi.8524749/


Processing page 5, link 23 of 30
https://nhattao.com/threads/co-so-san-xuat-moc-khoa-mica-moc-khoa-nhua-keo-mem-moc-khoa-qua-tang.8524438/


Processing page 5, link 24 of 30
https://nhattao.com/threads/chuyen-mua-tat-ca-cac-loai-xe-may-cu-ta

Processing page 7, link 24 of 30
https://nhattao.com/threads/chuyen-mua-tat-ca-cac-loai-xe-may-cu-tai-nha-gia-cao-nhat-sai-gon.6122465/


Processing page 7, link 25 of 30
https://nhattao.com/threads/muon-giao-luu-ip6-32g-voi-android.8430407/


Processing page 7, link 26 of 30
https://nhattao.com/threads/phan-mem-quan-ly-cong-no-tu-dong.8505203/


Processing page 7, link 27 of 30
https://nhattao.com/threads/tuyen-dung-viec-lam-tai-xe-b2-c-viec-lam-lo-xe-tai-giao-hang-tim-viec-lam-them-phu-kho-xep-hang-t.8491766/


Processing page 7, link 28 of 30
https://nhattao.com/threads/tai-sao-nen-mua-nick-cf-tai-shopnickre24h.8524110/


Processing page 7, link 29 of 30
https://nhattao.com/threads/nha-phan-phoi-quat-thong-gio-am-tran-nanyoo.8524109/


Processing page 7, link 30 of 30
https://nhattao.com/threads/trung-tam-thu-mua-xe-may-cu-gia-ca-canh-tranh.6122496/


Processing page 8 of 34
Processing page 8, link 1 of 30
https://nhattao.com/threads/shop-chuyen-thu-mua-may-choi-game-ds-2ds-3ds-psp-

Processing page 10, link 2 of 30
https://nhattao.com/threads/thu-mua-may-anh-cu-gia-cao-tan-noi-tai-ha-noi.8490455/


Processing page 10, link 3 of 30
https://nhattao.com/threads/can-mua-mot-so-linh-kien-may-tinh-bi-hu-hoac-loi.7296829/


Processing page 10, link 4 of 30
https://nhattao.com/threads/thu-mua-may-game-gia-cao-ps4-ps3-xbox-nintendo-switch-0947700818.6430441/


Processing page 10, link 5 of 30
https://nhattao.com/threads/ban-ten-mien-thegioiruouvang-com.8393484/


Processing page 10, link 6 of 30
https://nhattao.com/threads/ong-kinh-sigma-35mm-f-1-4-dg-hsm-art-for-canon.8524643/


Processing page 10, link 7 of 30
https://nhattao.com/threads/vi-nam.8525203/


Processing page 10, link 8 of 30
https://nhattao.com/threads/giay-nike-womens-air-zoom-vomero-13-running-shoe.8455852/


Processing page 10, link 9 of 30
https://nhattao.com/threads/in-bao-bi-nilon-dung-gao.8525386/


Processing page 10, link 10 of 30
https://nhattao.com/threads/ban-tai-khoan-google-drive-unlimited-goog

...

Processing page 25, link 24 of 30
https://nhattao.com/threads/chuyen-mua-tat-ca-cac-loai-xe-may-cu-tai-nha-gia-cao-nhat-sai-gon.6122465/


Processing page 25, link 25 of 30
https://nhattao.com/threads/muon-giao-luu-ip6-32g-voi-android.8430407/


Processing page 25, link 26 of 30
https://nhattao.com/threads/phan-mem-quan-ly-cong-no-tu-dong.8505203/


Processing page 25, link 27 of 30
https://nhattao.com/threads/tuyen-dung-viec-lam-tai-xe-b2-c-viec-lam-lo-xe-tai-giao-hang-tim-viec-lam-them-phu-kho-xep-hang-t.8491766/


Processing page 25, link 28 of 30
https://nhattao.com/threads/tai-sao-nen-mua-nick-cf-tai-shopnickre24h.8524110/


Processing page 25, link 29 of 30
https://nhattao.com/threads/nha-phan-phoi-quat-thong-gio-am-tran-nanyoo.8524109/


Processing page 25, link 30 of 30
https://nhattao.com/threads/trung-tam-thu-mua-xe-may-cu-gia-ca-canh-tranh.6122496/


Processing page 26 of 34
Processing page 26, link 1 of 30
https://nhattao.com/threads/thu-mua-laptop-cu-gia-cao-tphcm-09774198

Processing page 28 of 34
Processing page 28, link 1 of 30
https://nhattao.com/threads/thu-mua-laptop-cu-gia-cao-tphcm-0977419856-nghia.2597794/


Processing page 28, link 2 of 30
https://nhattao.com/threads/shop-chuyen-thu-mua-may-choi-game-ds-2ds-3ds-psp-vita-ps3-ps4-wiiu-xbox-077-707-1528.7220310/


Processing page 28, link 3 of 30
https://nhattao.com/threads/thu-mua-may-anh-cu-gia-cao-tan-noi-tai-ha-noi.8490455/


Processing page 28, link 4 of 30
https://nhattao.com/threads/can-mua-mot-so-linh-kien-may-tinh-bi-hu-hoac-loi.7296829/


Processing page 28, link 5 of 30
https://nhattao.com/threads/thu-mua-may-game-gia-cao-ps4-ps3-xbox-nintendo-switch-0947700818.6430441/


Processing page 28, link 6 of 30
https://nhattao.com/threads/ban-ten-mien-thegioiruouvang-com.8393484/


Processing page 28, link 7 of 30
https://nhattao.com/threads/ong-kinh-sigma-35mm-f-1-4-dg-hsm-art-for-canon.8524643/


Processing page 28, link 8 of 30
https://nhattao.com/threads/vi-nam.8525203/


Processing page 28

Processing page 30, link 8 of 30
https://nhattao.com/threads/vi-nam.8525203/


Processing page 30, link 9 of 30
https://nhattao.com/threads/giay-nike-womens-air-zoom-vomero-13-running-shoe.8455852/


Processing page 30, link 10 of 30
https://nhattao.com/threads/in-bao-bi-nilon-dung-gao.8525386/


Processing page 30, link 11 of 30
https://nhattao.com/threads/ban-tai-khoan-google-drive-unlimited-google-drive-khong-gioi-han.8366250/


Processing page 30, link 12 of 30
https://nhattao.com/threads/thanh-long-binh-thuan-gia-si.8495763/


Processing page 30, link 13 of 30
https://nhattao.com/threads/dich-vu-thu-mua-may-tinh-laptop-macbook-tai-ha-noi.8490457/


Processing page 30, link 14 of 30
https://nhattao.com/threads/oto-cu-thu-mua-oto-cu-gia-cao-tai-tp-hcm-mua-oto-da-qua-su-dung.3264194/


Processing page 30, link 15 of 30
https://nhattao.com/threads/mua-xe-dap-dien-cu-gia-cao.4403397/


Processing page 30, link 16 of 30
https://nhattao.com/threads/can-mua-con-ps-vita-tv.8423449/


Proce

Processing page 32, link 16 of 30
https://nhattao.com/threads/can-mua-con-ps-vita-tv.8423449/


Processing page 32, link 17 of 30
https://nhattao.com/threads/do-choi-xep-hinh-tu-chau-au-an-toan-thong-minh-kich-thich-phat-trien-tu-duy-cho-be.8466682/


Processing page 32, link 18 of 30
https://nhattao.com/threads/nhung-mau-thiep-sinh-nhat-vintage-vo-cung-doc-dao-va-la-mat.8524838/


Processing page 32, link 19 of 30
https://nhattao.com/threads/chuyen-thu-mua-laptop-cu-gia-cao-khu-vuc-tphcm.8494482/


Processing page 32, link 20 of 30
https://nhattao.com/threads/the-cao-data-3g-mobifone-tron-goi-1-nam.8517668/


Processing page 32, link 21 of 30
https://nhattao.com/threads/dich-vu-thu-mua-may-tinh-laptop-macbook-tai-ha-noi.8524751/


Processing page 32, link 22 of 30
https://nhattao.com/threads/thu-mua-may-anh-cu-gia-cao-tan-noi-tai-ha-noi.8524749/


Processing page 32, link 23 of 30
https://nhattao.com/threads/co-so-san-xuat-moc-khoa-mica-moc-khoa-nhua-keo-mem-moc-khoa-qua-tang.8524438/

Processing page 34, link 23 of 30
https://nhattao.com/threads/co-so-san-xuat-moc-khoa-mica-moc-khoa-nhua-keo-mem-moc-khoa-qua-tang.8524438/


Processing page 34, link 24 of 30
https://nhattao.com/threads/chuyen-mua-tat-ca-cac-loai-xe-may-cu-tai-nha-gia-cao-nhat-sai-gon.6122465/


Processing page 34, link 25 of 30
https://nhattao.com/threads/muon-giao-luu-ip6-32g-voi-android.8430407/


Processing page 34, link 26 of 30
https://nhattao.com/threads/phan-mem-quan-ly-cong-no-tu-dong.8505203/


Processing page 34, link 27 of 30
https://nhattao.com/threads/tuyen-dung-viec-lam-tai-xe-b2-c-viec-lam-lo-xe-tai-giao-hang-tim-viec-lam-them-phu-kho-xep-hang-t.8491766/


Processing page 34, link 28 of 30
https://nhattao.com/threads/tai-sao-nen-mua-nick-cf-tai-shopnickre24h.8524110/


Processing page 34, link 29 of 30
https://nhattao.com/threads/nha-phan-phoi-quat-thong-gio-am-tran-nanyoo.8524109/


Processing page 34, link 30 of 30
https://nhattao.com/threads/trung-tam-thu-mua-xe-may-cu-gia-ca-canh-t

In [8]:
test_pd = pd.read_json('./nhattao_marketplace.json')

In [9]:
test_pd.to_excel('./nhattao_marketplace.xlsx', index=False)

In [4]:
import requests, time, json, pandas as pd, numpy as np
from nhattao_crawler import Nhattao_crawler
from datetime import datetime
from bs4 import BeautifulSoup

In [5]:
# Marketplace page
url = 'https://nhattao.com/f/can-mua-trao-doi.589/'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) \
                AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

s = requests.Session()

In [6]:
page = s.get(url, headers=headers)
data = BeautifulSoup(page.text, 'html.parser')

In [7]:
data.find('div', class_='PageNav')['data-last']

'34'

### Class

In [ ]:
################################################################################################################################

In [1]:
import time, json, pandas as pd, numpy as np
from nhattao_crawler import Nhattao_crawler
from datetime import datetime

In [2]:
crawler = Nhattao_crawler('https://nhattao.com/f/can-mua-trao-doi.589/')
no_pages = crawler.get_no_pages()
combined = []

for i in range(1, no_pages+1):
    print('Processing page {} of {}'.format(i, no_pages))
       
    page_listings = crawler.get_listings_per_page(i)
    for j in range(len(page_listings)):
        sleep_timer = np.random.randint(5, 11)
        print('Processing page {} of {}, link {} of {}'.format(i, no_pages, \
                                                               j+1, len(page_listings)))
        print(page_listings[j])
        print('\n')
        
        crawler.process_data(page_listings[j])
        combined.append(crawler.process_data(page_listings[j]))
        
        # Sleep to prevent timeout
        print('Sleeping for {} seconds'.format(sleep_timer))
        time.sleep(sleep_timer)
print('Scrape completed')

Processing page 1 of 34
https://nhattao.com/f/can-mua-trao-doi.589/page-1
Processing page 1 of 34, link 1 of 30
https://nhattao.com/threads/dich-vu-thu-mua-may-tinh-laptop-macbook-tai-ha-noi.8490457/


Sleeping for 5 seconds
Processing page 1 of 34, link 2 of 30
https://nhattao.com/threads/shop-chuyen-thu-mua-may-choi-game-ds-2ds-3ds-psp-vita-ps3-ps4-wiiu-xbox-077-707-1528.7220310/


Sleeping for 5 seconds
Processing page 1 of 34, link 3 of 30
https://nhattao.com/threads/thu-mua-laptop-cu-gia-cao-tphcm-0977419856-nghia.2597794/


Sleeping for 5 seconds
Processing page 1 of 34, link 4 of 30
https://nhattao.com/threads/vien-giam-mo-bung-demar87-cell-belly-balance-co-tot-khong.8527611/


Sleeping for 5 seconds
Processing page 1 of 34, link 5 of 30
https://nhattao.com/threads/do-choi-xep-hinh-tu-chau-au-an-toan-thong-minh-kich-thich-phat-trien-tu-duy-cho-be.8466682/


Sleeping for 5 seconds
Processing page 1 of 34, link 6 of 30
https://nhattao.com/threads/ban-ten-mien-thegioiruouvang-com.8

Sleeping for 5 seconds
Processing page 2 of 34, link 27 of 30
https://nhattao.com/threads/ban-dat-mat-duong-ho-quy-ly-bim-son-thanh-hoa-gia-re.8522427/


Sleeping for 5 seconds
Processing page 2 of 34, link 28 of 30
https://nhattao.com/threads/dat-nen-so-do-ven-bien-ca-na-ninh-thuan.8522335/


Sleeping for 5 seconds
Processing page 2 of 34, link 29 of 30
https://nhattao.com/threads/ban-nhanh-mat-tien-phan-dang-luu-pn-ngang-6-no-hau-9m-gia-37-ty.8522120/


Sleeping for 5 seconds
Processing page 2 of 34, link 30 of 30
https://nhattao.com/threads/50-ty-so-huu-ngay-nha-dep-mt-hoa-lan-phan-xich-long-12x16m-50-ty.8522116/


Sleeping for 5 seconds
Processing page 3 of 34
https://nhattao.com/f/can-mua-trao-doi.589/page-3
Processing page 3 of 34, link 1 of 30
https://nhattao.com/threads/giay-the-thao-champion.8521573/


Sleeping for 5 seconds
Processing page 3 of 34, link 2 of 30
https://nhattao.com/threads/di-du-hoc-nhat-ban-co-tot-hay-khong.8521432/


Sleeping for 5 seconds
Processing page 3 

Sleeping for 5 seconds
Processing page 4 of 34, link 22 of 30
https://nhattao.com/threads/top-3-mau-mixer-co-am-thanh-tot-nhat-ua-chuong-nhat-hien-nay.8516044/


Sleeping for 5 seconds
Processing page 4 of 34, link 23 of 30
https://nhattao.com/threads/ps-vita-2000-32g-hack-full.8515018/


Sleeping for 5 seconds
Processing page 4 of 34, link 24 of 30
https://nhattao.com/threads/can-mua-sac-cho-palm-treo-650.8515906/


Sleeping for 5 seconds
Processing page 4 of 34, link 25 of 30
https://nhattao.com/threads/ban-kinh-bao-ho-lao-dong-sm911-tai-ha-giang.8515851/


Sleeping for 5 seconds
Processing page 4 of 34, link 26 of 30
https://nhattao.com/threads/cac-loai-thanh-ray-gan-tuong-va-cong-dung-ma-ban-khong-ngo.8515740/


Sleeping for 5 seconds
Processing page 4 of 34, link 27 of 30
https://nhattao.com/threads/xuong-san-xuat-bia-da-chuyen-cung-cap-cuon-menu-lam-bia-dung-ho-so-bia-cong-bia-kep-hoa-don.8515727/


Sleeping for 5 seconds
Processing page 4 of 34, link 28 of 30
https://nhattao.com

Sleeping for 5 seconds
Processing page 6 of 34, link 17 of 30
https://nhattao.com/threads/thao-plastic-ep-deo-ep-nhua-bang-may-moc.8508734/


Sleeping for 5 seconds
Processing page 6 of 34, link 18 of 30
https://nhattao.com/threads/thu-mua-may-game-cu-thu-mua-ps3-thu-mua-ps4-thu-mua-psp-mua-vita-cu-tan-noi-gia-cao.8131437/


Sleeping for 5 seconds
Processing page 6 of 34, link 19 of 30
https://nhattao.com/threads/can-mua-may-hoac-xac-sony-j10i2-hoac-j10i.8508374/


Sleeping for 5 seconds
Processing page 6 of 34, link 20 of 30
https://nhattao.com/threads/guitar-eko-evo-iii-baritone-blend-eq-nk-y.8508138/


Sleeping for 5 seconds
Processing page 6 of 34, link 21 of 30
https://nhattao.com/threads/guitar-acoustic-eko-evo-iii-018-cw-blend-eq-cua-italya.8508136/


Sleeping for 5 seconds
Processing page 6 of 34, link 22 of 30
https://nhattao.com/threads/may-ao-thun-dong-phuc-tp-hcm.8508041/


Sleeping for 5 seconds
Processing page 6 of 34, link 23 of 30
https://nhattao.com/threads/apple-watch

Sleeping for 5 seconds
Processing page 8 of 34, link 11 of 30
https://nhattao.com/threads/thu-mua-ipad-iphone-cu-gia-cao-09-3462899-8.8501948/


Sleeping for 5 seconds
Processing page 8 of 34, link 12 of 30
https://nhattao.com/threads/nhung-kich-thuoc-chuan-cua-tham-dinh-bui.8501941/


Sleeping for 5 seconds
Processing page 8 of 34, link 13 of 30
https://nhattao.com/threads/chuyen-luyen-thi-ielts-cap-toc.8500549/


Sleeping for 5 seconds
Processing page 8 of 34, link 14 of 30
https://nhattao.com/threads/du-an-centa-city-hai-phong.8500456/


Sleeping for 5 seconds
Processing page 8 of 34, link 15 of 30
https://nhattao.com/threads/zobet88-com-giao-mang-tong-master-agent-va-member-bong88-sbobet-3in1bet-s128-da-ga-sv388.8500064/


Sleeping for 5 seconds
Processing page 8 of 34, link 16 of 30
https://nhattao.com/threads/7-tam-thiep-sinh-nhat-ban-trai-kem-theo-loi-chuc-ngot-ngao-nhat.8499972/


Sleeping for 5 seconds
Processing page 8 of 34, link 17 of 30
https://nhattao.com/threads/the-tag-

...

Sleeping for 5 seconds
Processing page 31 of 34, link 97 of 97
https://nhattao.com/threads/can-mua-xac-iphone-5-or-5s.8190983/


Sleeping for 5 seconds
Processing page 32 of 34
https://nhattao.com/f/can-mua-trao-doi.589/page-32
Processing page 32 of 34, link 1 of 88
https://nhattao.com/threads/tour-du-lich-ha-giang.8429372/


Sleeping for 5 seconds
Processing page 32 of 34, link 2 of 88
https://nhattao.com/threads/top-5-mau-kinh-mat-nam-thoi-trang-dep-chat-luong-nhat-nam-2019.8429356/


Sleeping for 5 seconds
Processing page 32 of 34, link 3 of 88
https://nhattao.com/threads/ban-cho-phoc-soc-thuan-chung-dogily-petshop.8429346/


Sleeping for 5 seconds
Processing page 32 of 34, link 4 of 88
https://nhattao.com/threads/da-co-sac-rt.8428351/


Sleeping for 5 seconds
Processing page 32 of 34, link 5 of 88
https://nhattao.com/threads/vong-deo-tay-go-dan-huong-mexico.8428849/


Sleeping for 5 seconds
Processing page 32 of 34, link 6 of 88
https://nhattao.com/threads/thu-mua-laptop-hu-cu-be-t

Sleeping for 5 seconds
Processing page 32 of 34, link 57 of 88
https://nhattao.com/threads/do-mat-tim-nguoi-bao-ve-dip-tet.8422902/


Sleeping for 5 seconds
Processing page 32 of 34, link 58 of 88
https://nhattao.com/threads/cung-cap-luoi-do-bong-thuy-tinh-d1a100x100-gia-re-nhat.8422885/


Sleeping for 5 seconds
Processing page 32 of 34, link 59 of 88
https://nhattao.com/threads/tivi-asanzo-tivi-gia-re-nhat-thi-truong-viet-nam.8422554/


Sleeping for 5 seconds
Processing page 32 of 34, link 60 of 88
https://nhattao.com/threads/chung-toi-chuyen-nhan-lap-dat-xay-ban-mo-da-tai-dak-lak-uy-tin-chat-luong.8422493/


Sleeping for 5 seconds
Processing page 32 of 34, link 61 of 88
https://nhattao.com/threads/ngan-hang-can-thanh-ly-3-nha-pho-lien-ke-ho-tro-vay-den-60.8422332/


Sleeping for 5 seconds
Processing page 32 of 34, link 62 of 88
https://nhattao.com/threads/nhan-thiet-ke-lap-dat-mo-da-va-khu-lang-mo-da-tai-ninh-thuan.8422156/


Sleeping for 5 seconds
Processing page 32 of 34, link 63 o

Sleeping for 5 seconds
Processing page 33 of 34, link 25 of 88
https://nhattao.com/threads/3-chu-y-quan-trong-de-kien-co-trung-tuyen-khi-xet-hoc-ba-dh-cd-2019.8426160/


Sleeping for 5 seconds
Processing page 33 of 34, link 26 of 88
https://nhattao.com/threads/phan-mem-nuoi-nick-tren-dien-thoai-chat-luong.8425941/


Sleeping for 5 seconds
Processing page 33 of 34, link 27 of 88
https://nhattao.com/threads/phan-mem-ket-ban-facebook-hang-loat-5000-ban-be.8425937/


Sleeping for 5 seconds
Processing page 33 of 34, link 28 of 88
https://nhattao.com/threads/ban-kiot-khu-do-thi-thanh-ha.8425628/


Sleeping for 5 seconds
Processing page 33 of 34, link 29 of 88
https://nhattao.com/threads/mau-cot-da-vuong-cho-nha-tho-ho-tot-nhat-hien-nay.8425595/


Sleeping for 5 seconds
Processing page 33 of 34, link 30 of 88
https://nhattao.com/threads/mau-mo-da-don-gian-dep-cham-khac-tinh-te-do-ben-cao.8425540/


Sleeping for 5 seconds
Processing page 33 of 34, link 31 of 88
https://nhattao.com/threads/8425

Sleeping for 5 seconds
Processing page 33 of 34, link 81 of 88
https://nhattao.com/threads/phan-mem-quang-cao-dang-tin-ban-hang-tren-facebook.8417568/


Sleeping for 5 seconds
Processing page 33 of 34, link 82 of 88
https://nhattao.com/threads/hn-can-mua-apple-watch-ban-gom-size-38-user.8417220/


Sleeping for 5 seconds
Processing page 33 of 34, link 83 of 88
https://nhattao.com/threads/cv-xin-viec-star-cv-cua-su-khac-biet-va-cua-su-thanh-cong.8414392/


Sleeping for 5 seconds
Processing page 33 of 34, link 84 of 88
https://nhattao.com/threads/can-mua-loa-jbl-charge-3-chinh-hang-cu.8412713/


Sleeping for 5 seconds
Processing page 33 of 34, link 85 of 88
https://nhattao.com/threads/thu-mua-xac-iphone-x-thu-mua-xac-iphone-xs-thu-mua-xac-iphone-xs-max.7922838/


Sleeping for 5 seconds
Processing page 33 of 34, link 86 of 88
https://nhattao.com/threads/thu-mua-xac-dien-thoai.2961885/


Sleeping for 5 seconds
Processing page 33 of 34, link 87 of 88
https://nhattao.com/threads/can-giao-luu-

Sleeping for 5 seconds
Processing page 34 of 34, link 50 of 88
https://nhattao.com/threads/ep-man-ipad-4.8423796/


Sleeping for 5 seconds
Processing page 34 of 34, link 51 of 88
https://nhattao.com/threads/dia-chi-mua-bep-tu-taka-chinh-hang.8423783/


Sleeping for 5 seconds
Processing page 34 of 34, link 52 of 88
https://nhattao.com/threads/nhan-thi-cong-lap-dat-xay-mo-da-tai-ha-tinh-uy-tin-chat-luong.8423725/


Sleeping for 5 seconds
Processing page 34 of 34, link 53 of 88
https://nhattao.com/threads/giao-luu-5s-hoac-se.8423687/


Sleeping for 5 seconds
Processing page 34 of 34, link 54 of 88
https://nhattao.com/threads/tim-mua-mi-8-6-128-den-moi-hoac-cu.8423680/


Sleeping for 5 seconds
Processing page 34 of 34, link 55 of 88
https://nhattao.com/threads/nem-cao-su-tong-hop-gap-2-kim-cuong-160x200x10cm.8423653/


Sleeping for 5 seconds
Processing page 34 of 34, link 56 of 88
https://nhattao.com/threads/e-co-mate-20-pro-chinh-hang-con-bao-hanh-12-2019-muon-gl-sang-note9-a.8423548/


S

In [3]:
print('Writing json file')
with open('../data/'+str(datetime.timestamp(datetime.now()))+'_nhattao_marketplace.json', 'w') as f:
    json.dump(combined, f)

Writing json file


In [5]:
test_pd = pd.read_json('../data/1569598808.878166_nhattao_marketplace.json')
test_pd.to_excel('../data/'+str(datetime.timestamp(datetime.now()))+'_nhattao_marketplace.xlsx', \
                 index=False)

In [ ]:
################################################################################################################################

In [2]:
crawler = Nhattao_crawler('https://nhattao.com/f/dien-thoai.543/')

no_pages = crawler.get_no_pages()
combined = []

for i in range(1, no_pages+1):
    print('Processing page {} of {}'.format(i, no_pages))
       
    page_listings = crawler.get_listings_per_page(i)
    for j in range(len(page_listings)):
        sleep_timer = np.random.randint(5, 11)
        print('Processing page {} of {}, link {} of {}'.format(i, no_pages, \
                                                               j+1, len(page_listings)))
        print(page_listings[j])
        print('\n')
        
        crawler.process_data(page_listings[j])
        combined.append(crawler.process_data(page_listings[j]))
        
        # Sleep to prevent timeout
        print('Sleeping for {} seconds'.format(sleep_timer))
        time.sleep(sleep_timer)
print('Scrape completed')

Processing page 1 of 34
https://nhattao.com/f/dien-thoai.543/page-1
Processing page 1 of 34, link 1 of 30
https://nhattao.com/threads/sac-du-phong-iwalk-12000mah-ubt12000x.8482923/


Sleeping for 10 seconds
Processing page 1 of 34, link 2 of 30
https://nhattao.com/threads/lg-g6-han-ban-64gb-may-dep-nhu-moi-2tr3.8468355/


Sleeping for 9 seconds
Processing page 1 of 34, link 3 of 30
https://nhattao.com/threads/note-8-note-9-s9-s9-s10-s10-s105g.8405004/


Sleeping for 9 seconds
Processing page 1 of 34, link 4 of 30
https://nhattao.com/threads/note9-512gb-vn-moi-nguyen-siu.8522106/


Sleeping for 7 seconds
Processing page 1 of 34, link 5 of 30
https://nhattao.com/threads/xac-song-hong-pin.8527884/


Sleeping for 10 seconds
Processing page 1 of 34, link 6 of 30
https://nhattao.com/threads/iphone-x-256g-den-iphone-xs-max-64g-vn-a-gold.8527398/


Sleeping for 8 seconds
Processing page 1 of 34, link 7 of 30
https://nhattao.com/threads/samsung-note10-5g-s10plus-s105g-s10e-s10-note9-note8-s8-s8

Sleeping for 10 seconds
Processing page 2 of 34, link 26 of 30
https://nhattao.com/threads/ban-lai-ch-hung-phat-1-le-van-luong-phuoc-kien-nha-be-2-pn-84m2-1-750-ty.8522535/


Sleeping for 6 seconds
Processing page 2 of 34, link 27 of 30
https://nhattao.com/threads/ban-dat-mat-duong-ho-quy-ly-bim-son-thanh-hoa-gia-re.8522427/


Sleeping for 7 seconds
Processing page 2 of 34, link 28 of 30
https://nhattao.com/threads/dat-nen-so-do-ven-bien-ca-na-ninh-thuan.8522335/


Sleeping for 10 seconds
Processing page 2 of 34, link 29 of 30
https://nhattao.com/threads/ban-nhanh-mat-tien-phan-dang-luu-pn-ngang-6-no-hau-9m-gia-37-ty.8522120/


Sleeping for 9 seconds
Processing page 2 of 34, link 30 of 30
https://nhattao.com/threads/50-ty-so-huu-ngay-nha-dep-mt-hoa-lan-phan-xich-long-12x16m-50-ty.8522116/


Sleeping for 7 seconds
Processing page 3 of 34
https://nhattao.com/f/dien-thoai.543/page-3
Processing page 3 of 34, link 1 of 30
https://nhattao.com/threads/giay-the-thao-champion.8521573/


Sleepin

Sleeping for 9 seconds
Processing page 4 of 34, link 21 of 30
https://nhattao.com/threads/ban-cup-the-thao-cac-loai-mon-the-thao-bong-da-bong-chuyen-tennis-golf-cau-long-bong-ban.8516051/


Sleeping for 8 seconds
Processing page 4 of 34, link 22 of 30
https://nhattao.com/threads/top-3-mau-mixer-co-am-thanh-tot-nhat-ua-chuong-nhat-hien-nay.8516044/


Sleeping for 6 seconds
Processing page 4 of 34, link 23 of 30
https://nhattao.com/threads/ps-vita-2000-32g-hack-full.8515018/


Sleeping for 9 seconds
Processing page 4 of 34, link 24 of 30
https://nhattao.com/threads/can-mua-sac-cho-palm-treo-650.8515906/


Sleeping for 7 seconds
Processing page 4 of 34, link 25 of 30
https://nhattao.com/threads/ban-kinh-bao-ho-lao-dong-sm911-tai-ha-giang.8515851/


Sleeping for 10 seconds
Processing page 4 of 34, link 26 of 30
https://nhattao.com/threads/cac-loai-thanh-ray-gan-tuong-va-cong-dung-ma-ban-khong-ngo.8515740/


Sleeping for 8 seconds
Processing page 4 of 34, link 27 of 30
https://nhattao.com/t

Processing page 6 of 34, link 15 of 30
https://nhattao.com/threads/can-pin-may-tinh-bang-dell-venue-8.8508895/


Sleeping for 8 seconds
Processing page 6 of 34, link 16 of 30
https://nhattao.com/threads/xuong-may-bia-simili-san-xuat-bia-trinh-ky-doi-bia-folder-dong-cong.8012362/


Sleeping for 6 seconds
Processing page 6 of 34, link 17 of 30
https://nhattao.com/threads/thao-plastic-ep-deo-ep-nhua-bang-may-moc.8508734/


Sleeping for 5 seconds
Processing page 6 of 34, link 18 of 30
https://nhattao.com/threads/thu-mua-may-game-cu-thu-mua-ps3-thu-mua-ps4-thu-mua-psp-mua-vita-cu-tan-noi-gia-cao.8131437/


Sleeping for 6 seconds
Processing page 6 of 34, link 19 of 30
https://nhattao.com/threads/can-mua-may-hoac-xac-sony-j10i2-hoac-j10i.8508374/


Sleeping for 7 seconds
Processing page 6 of 34, link 20 of 30
https://nhattao.com/threads/guitar-eko-evo-iii-baritone-blend-eq-nk-y.8508138/


Sleeping for 7 seconds
Processing page 6 of 34, link 21 of 30
https://nhattao.com/threads/guitar-acoustic

Sleeping for 8 seconds
Processing page 8 of 34, link 9 of 30
https://nhattao.com/threads/thiet-ke-web-chuan-seo-2019-tai-tphcm.8502470/


Sleeping for 10 seconds
Processing page 8 of 34, link 10 of 30
https://nhattao.com/threads/mau-mo-to-bang-da-dep-nhat-hien-nay.8502108/


Sleeping for 8 seconds
Processing page 8 of 34, link 11 of 30
https://nhattao.com/threads/thu-mua-ipad-iphone-cu-gia-cao-09-3462899-8.8501948/


Sleeping for 9 seconds
Processing page 8 of 34, link 12 of 30
https://nhattao.com/threads/nhung-kich-thuoc-chuan-cua-tham-dinh-bui.8501941/


Sleeping for 9 seconds
Processing page 8 of 34, link 13 of 30
https://nhattao.com/threads/chuyen-luyen-thi-ielts-cap-toc.8500549/


Sleeping for 8 seconds
Processing page 8 of 34, link 14 of 30
https://nhattao.com/threads/du-an-centa-city-hai-phong.8500456/


Sleeping for 5 seconds
Processing page 8 of 34, link 15 of 30
https://nhattao.com/threads/zobet88-com-giao-mang-tong-master-agent-va-member-bong88-sbobet-3in1bet-s128-da-ga-sv38

...

Sleeping for 5 seconds
Processing page 23 of 34, link 3 of 30
https://nhattao.com/threads/dia-chi-ban-mo-cong-giao-bang-da-dep-chat-luong-gia-re-tai-ninh-binh.8433759/


Sleeping for 8 seconds
Processing page 23 of 34, link 4 of 30
https://nhattao.com/threads/lua-chon-ghe-massage-nhu-the-nao-cho-hop-ly.8433689/


Sleeping for 5 seconds
Processing page 23 of 34, link 5 of 30
https://nhattao.com/threads/thuong-hieu-tresette-han-quoc-chuyen-nghien-cuu-va-phat-trien-dong-san-pham-balo-tui-xach.8432730/


Sleeping for 10 seconds
Processing page 23 of 34, link 6 of 30
https://nhattao.com/threads/ban-gap-can-nha.8433649/


Sleeping for 10 seconds
Processing page 23 of 34, link 7 of 30
https://nhattao.com/threads/can-mua-iphone-6s-hoac-7-mat-van-tay.8432567/


Sleeping for 7 seconds
Processing page 23 of 34, link 8 of 30
https://nhattao.com/threads/mua-hop-galaxy-busd.8433362/


Sleeping for 9 seconds
Processing page 23 of 34, link 9 of 30
https://nhattao.com/threads/master-batch-duoc-ung-dung

Sleeping for 10 seconds
Processing page 24 of 34, link 27 of 30
https://nhattao.com/threads/thu-mua-may-anh-cu-0941777111.8428256/


Sleeping for 5 seconds
Processing page 24 of 34, link 28 of 30
https://nhattao.com/threads/can-mua-pokemon-black-white.8428251/


Sleeping for 5 seconds
Processing page 24 of 34, link 29 of 30
https://nhattao.com/threads/long-den-trung-thu-quang-cao-so-luong-lon.8427941/


Sleeping for 5 seconds
Processing page 24 of 34, link 30 of 30
https://nhattao.com/threads/giay-dr-martens-nam-mau-nau-nhe.8427082/


Sleeping for 10 seconds
Processing page 25 of 34
https://nhattao.com/f/dien-thoai.543/page-25
Processing page 25 of 34, link 1 of 30
https://nhattao.com/threads/thu-mua-ps4-cu-hong-ps3-slim-cu-ps-vita-nintendo-switch-ps2-slim-psp-cu-o-tai-ha-noi.8106786/


Sleeping for 7 seconds
Processing page 25 of 34, link 2 of 30
https://nhattao.com/threads/can-mua-cpu-intel-core-2-duo-t9500-hoac-intel-core-2-extreme-x9000.8427259/


Sleeping for 5 seconds
Processing 

Sleeping for 9 seconds
Processing page 26 of 34, link 22 of 30
https://nhattao.com/threads/long-bien-can-mua-samsung-galaxy-s7e-2-sim.8421343/


Sleeping for 7 seconds
Processing page 26 of 34, link 23 of 30
https://nhattao.com/threads/nhung-tieu-chuan-phong-sach-cua-benh-vien.8421331/


Sleeping for 7 seconds
Processing page 26 of 34, link 24 of 30
https://nhattao.com/threads/chinh-chu-can-ban-hoac-cho-thue-nha-tinh-gia-thanh-hoa.8421235/


Sleeping for 6 seconds
Processing page 26 of 34, link 25 of 30
https://nhattao.com/threads/dien-may-hoang-long-cung-cap-may-phat-dien-cho-du-an.8421201/


Sleeping for 7 seconds
Processing page 26 of 34, link 26 of 30
https://nhattao.com/threads/ban-dia-ps4-monster-hunter-world.8421025/


Sleeping for 9 seconds
Processing page 26 of 34, link 27 of 30
https://nhattao.com/threads/0934-644-179-thu-mua-pallet-nhua-cu-gia-cao-thu-mua-pallet-nhu-cu-gia-cao.8420962/


Sleeping for 5 seconds
Processing page 26 of 34, link 28 of 30
https://nhattao.com/threa

Processing page 28 of 34, link 16 of 30
https://nhattao.com/threads/mgate-mb3180.8414569/


Sleeping for 6 seconds
Processing page 28 of 34, link 17 of 30
https://nhattao.com/threads/cv-xin-viec-star-cv-cua-su-khac-biet-va-cua-su-thanh-cong.8414392/


Sleeping for 10 seconds
Processing page 28 of 34, link 18 of 30
https://nhattao.com/threads/sofa-bed-co-kich-thuoc-1m80-x-90.8414386/


Sleeping for 9 seconds
Processing page 28 of 34, link 19 of 30
https://nhattao.com/threads/nhan-thi-cong-lap-dat-xay-mo-da-o-bac-ninh-uy-tin-chat-luong.8414368/


Sleeping for 5 seconds
Processing page 28 of 34, link 20 of 30
https://nhattao.com/threads/vitme-tbi.8414358/


Sleeping for 10 seconds
Processing page 28 of 34, link 21 of 30
https://nhattao.com/threads/chinh-sach-bao-hanh-doi-voi-chu-ky-so-viettel.8414353/


Sleeping for 7 seconds
Processing page 28 of 34, link 22 of 30
https://nhattao.com/threads/buc-binh-phong-nha-tho-ho-bang-da-cham-khac-hoa-van-tinh-te.8413914/


Sleeping for 5 seconds
Pro

Sleeping for 5 seconds
Processing page 30 of 34, link 11 of 30
https://nhattao.com/threads/thanh-rang-cheo-mini.8407336/


Sleeping for 8 seconds
Processing page 30 of 34, link 12 of 30
https://nhattao.com/threads/phan-mem-ban-hang-shop-posxanh.8407332/


Sleeping for 9 seconds
Processing page 30 of 34, link 13 of 30
https://nhattao.com/threads/ninja-auto-post-phan-mem-lam-chu-thoi-gian-quang-cao.8407262/


Sleeping for 5 seconds
Processing page 30 of 34, link 14 of 30
https://nhattao.com/threads/da-ai-mua-hang-tai-vu-gia-phat-chua.8407248/


Sleeping for 5 seconds
Processing page 30 of 34, link 15 of 30
https://nhattao.com/threads/phan-mem-tu-dong-tra-loi-facebook-phan-mem-quan-ly-comment-inbox-tot-nhat.8407219/


Sleeping for 5 seconds
Processing page 30 of 34, link 16 of 30
https://nhattao.com/threads/lua-chon-dang-ky-goi-cuoc-combo-truyen-hinh-va-cap-quang-viettel-tiet-kiem-nhat.8407213/


Sleeping for 6 seconds
Processing page 30 of 34, link 17 of 30
https://nhattao.com/threads/ph

In [3]:
print('Writing json file')
with open('../data/'+str(datetime.timestamp(datetime.now()))+'_nhattao_phone.json', 'w') as f:
    json.dump(combined, f)

Writing json file


In [4]:
test_pd = pd.read_json('../data/1569608621.072634_nhattao_phone.json')
test_pd.to_excel('../data/'+str(datetime.timestamp(datetime.now()))+'_nhattao_phone.xlsx', \
                 index=False)

In [ ]:
################################################################################################################################

In [ ]:
crawler = Nhattao_crawler('https://nhattao.com/f/may-tinh-bang.548/')

no_pages = crawler.get_no_pages()
combined = []

for i in range(1, no_pages+1):
    print('Processing page {} of {}'.format(i, no_pages))
       
    page_listings = crawler.get_listings_per_page(i)
    for j in range(len(page_listings)):
        sleep_timer = np.random.randint(5, 11)
        print('Processing page {} of {}, link {} of {}'.format(i, no_pages, \
                                                               j+1, len(page_listings)))
        print(page_listings[j])
        print('\n')
        
        crawler.process_data(page_listings[j])
        combined.append(crawler.process_data(page_listings[j]))
        
        # Sleep to prevent timeout
        print('Sleeping for {} seconds'.format(sleep_timer))
        time.sleep(sleep_timer)
print('Scrape completed')

In [ ]:
print('Writing json file')
with open('../data/'+str(datetime.timestamp(datetime.now()))+'_nhattao_tablet.json', 'w') as f:
    json.dump(combined, f)

In [ ]:
test_pd = pd.read_json('./nhattao_marketplace.json')
test_pd.to_excel('../data/'+str(datetime.timestamp(datetime.now()))+'_nhattao_phone.xlsx', \
                 index=False)

In [ ]:
################################################################################################################################

### Chotot

In [2]:
import time, requests, time, numpy as np, pandas as pd
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup

In [11]:
# Set main url page
url = 'https://nha.chotot.com/#regionselect?v=1.1'

# Set driver for selenium
driver = webdriver.Chrome('./chromedriver')

In [12]:
# Retrieve webpage
driver.get(url)

In [ ]:
# Set main url page
url = 'https://nha.chotot.com/#regionselect?v=1.1'

# Set driver for selenium
driver = webdriver.Chrome('./chromedriver')

# Retrieve webpage
driver.get(url)

# Parse retrieved page 
page_links = BeautifulSoup(driver.page_source, 'html.parser')

# Retrieve all page links
pages = set()
for page in page_links.find_all('li', class_='sc-dVhcbM cZLfdN'):
    pages.add(page.a.get('href'))
pages = list(pages)

# Cycle through each page link
combined = []
for i in range(0, len(pages)+1):
    print('Processing page {} of {}'.format(i+1, len(pages)))
    print('Retrieving links')
    
    links = []
    page_url = 'https://nha.chotot.com'+pages[i]    
    driver.get(url)   
    
    post_links = BeautifulSoup(driver.page_source, 'html.parser')

    for link in post_links.find_all('a', class_='_3JMKvS6hucA6KaM9tX3Qb1'):
        links.append('https://nha.chotot.com'+link.get('href'))
        
    for link in links:
        driver.get(link)
        driver.find_element_by_xpath('//*[@id="app"]/div[2]/main/article/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div[1]/div/div').click()
        posting_info = BeautifulSoup(driver.page_source, 'html.parser')
        results = {
            'Seller': posting_info.find('div', class_='sc-eilVRo hEXVti').text,
            'Seller Type': posting_info.find('div', class_='sc-fhYwyz fDMLIV').text,
            'Chat Response': posting_info.find('div', class_='sc-fhYwyz fDMLIV').span.text,
            'Contact': posting_info.find('div', class_='sc-caSCKo beAliT').text,
            'Title': posting_info.find('h1', class_='_22kG1zbJ4D-6IUEgKvoifC col-xs-12').text,
            'Price': posting_info.find('span', class_='oRSYZ0HPb2tHhHjpVp_2o').\
                    text.replace(' VND', '').replace('.', ''),
            'Area': posting_info.find('h1', class_='_22kG1zbJ4D-6IUEgKvoifC col-xs-12').text,
        }

In [6]:
# Set main url page
url = 'https://nha.chotot.com/#regionselect?v=1.1'

# Set driver for selenium
driver = webdriver.Chrome('./chromedriver')

# Retrieve webpage
driver.get(url)

time.sleep(6)

# Parse retrieved page 
page_links = BeautifulSoup(driver.page_source, 'html.parser')

# Retrieve all page links
pages = set()
for page in page_links.find_all('li', class_='sc-dVhcbM cZLfdN'):
    pages.add(page.a.get('href'))
pages = list(pages)
print(pages)

time.sleep(5)

# Cycle through each page link
links = []
page_url = 'https://nha.chotot.com'+pages[0]    
driver.get(url)   
    
post_links = BeautifulSoup(driver.page_source, 'html.parser')

for link in post_links.find_all('a', class_='_3JMKvS6hucA6KaM9tX3Qb1'):
    links.append('https://nha.chotot.com'+link.get('href'))

print(links)    

driver.get(links[0])
time.sleep(5)

with open('./posting.html', 'w') as f:
        f.write(driver.page_source)

driver.quit()
# posting_info = BeautifulSoup(driver.page_source, 'html.parser')

['/?page=5', '/?page=6', '/?page=3', '/?page=4', '/?page=1', '/?page=9', '/?page=8', '/?page=7', '/?page=2']
['https://nha.chotot.com/ha-noi/quan-ha-dong/mua-ban-nha-dat/63392963.htm', 'https://nha.chotot.com/tp-ho-chi-minh/quan-go-vap/sang-nhuong-van-phong-mat-bang-kinh-doanh/63392961.htm', 'https://nha.chotot.com/ninh-thuan/thanh-pho-phan-rang-thap-cham/mua-ban-dat/63392962.htm', 'https://nha.chotot.com/tp-ho-chi-minh/quan-10/mua-ban-nha-dat/63392957.htm', 'https://nha.chotot.com/tp-ho-chi-minh/quan-go-vap/mua-ban-nha-dat/63392956.htm', 'https://nha.chotot.com/tp-ho-chi-minh/quan-tan-phu/sang-nhuong-van-phong-mat-bang-kinh-doanh/63392955.htm', 'https://nha.chotot.com/tp-ho-chi-minh/quan-go-vap/mua-ban-nha-dat/56226848.htm', 'https://nha.chotot.com/tien-giang/huyen-chau-thanh/mua-ban-nha-dat/63392951.htm', 'https://nha.chotot.com/tp-ho-chi-minh/huyen-cu-chi/mua-ban-dat/63392950.htm', 'https://nha.chotot.com/tra-vinh/thanh-pho-tra-vinh/mua-ban-dat/63392952.htm', 'https://nha.chotot.com

In [24]:
j = BeautifulSoup(open('./posting.html', 'r'), 'html.parser')

j.find('a', class_='H3QBvet3qzdHlB3LVAw-7 btn btn-success hidden-sm hidden-md hidden-lg hidden-xl')['href'].split(':')[1]

'0392017964'